### Yellow Taxi Trip Records

In [1]:
#Importar librerías.
import numpy as np
import pandas as pd

In [2]:
#Leer diccionario con vínculos.
diccionario = pd.read_pickle('datos_tlc/tlc_amarillos')

In [3]:
#Descargar primer tabla (12/2018).
exec(diccionario['yellow_2018_61'])

In [4]:
#Ver primer tabla (12/2018).
yellow_2018_61.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8195675 entries, 0 to 8195674
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   object        
 18  airport_fee           

Considerar eliminar:
- VendorID (Código que indica el proveedor de datos)
- RateCodeID (Código de tarifa final en vigor al final del viaje)
- Store_and_fwd_flag (Indica si el registro del viaje se mantuvo en memoria del vehículo antes de enviarlo al proveedor)

In [5]:
#Eliminar columnas indeseadas.
yellow_2018_61 = yellow_2018_61.drop(columns = ['VendorID', 'RatecodeID', 'store_and_fwd_flag'])

Separar:
- tpep_pickup_datetime
- tpep_dropoff_datetime

En día (YYYY-MM-DD)
Hora de pickup
Hora de dropoff

Son tipo datetime64 (numpy)

In [6]:
#Insertar 'pickup_day'.
yellow_2018_61.insert(0, 'pickup_day', yellow_2018_61['tpep_pickup_datetime'].dt.date)

In [7]:
#Insertar 'pickup_time'.
yellow_2018_61.insert(1, 'pickup_time', yellow_2018_61['tpep_pickup_datetime'].dt.time)

In [8]:
#Insertar 'dropoff_day'.
yellow_2018_61.insert(2, 'dropoff_day', yellow_2018_61['tpep_dropoff_datetime'].dt.date)

In [9]:
#Insertar 'dropoff_time'.
yellow_2018_61.insert(3, 'dropoff_time', yellow_2018_61['tpep_dropoff_datetime'].dt.time)

In [10]:
#Eliminar 'tpep_pickup_datetime' y 'tpep_dropoff_datetime'.
yellow_2018_61 = yellow_2018_61.drop(columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [11]:
#Comparar 'pickup_day' y 'dropoff_day'.
yellow_2018_61['pickup_day'].compare(yellow_2018_61['dropoff_day'])

,self,other
214,2018-11-30,2018-12-01
353,2018-11-30,2018-12-01
382,2018-11-30,2018-12-01
468,2018-11-30,2018-12-01
564,2018-11-30,2018-12-01
...,...,...
8194384,2018-12-29,2018-12-30
8194385,2018-12-29,2018-12-30
8194386,2018-12-29,2018-12-30
8194953,2018-12-30,2018-12-31


Ya que hay bastantes diferencias, se deja de esa forma.

Identificar valores faltantes

In [12]:
#Encontrar null.
yellow_2018_61.isnull().sum()

pickup_day                     0
pickup_time                    0
dropoff_day                    0
dropoff_time                   0
passenger_count            22188
trip_distance                  0
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     8195675
airport_fee              8195675
dtype: int64

No existen datos en esta tabla de 'congestion_surcharge' o 'airport_fee', pero las columnas se mantendrán y se cambiarán sus valores nulos por '0' por si en otros meses existen estos cobros adicionales al pasajero.

In [13]:
#Sustituir nulos en 'congestion_surcharge' y 'airport_fee' por '0'.
yellow_2018_61[['congestion_surcharge', 'airport_fee']] = yellow_2018_61[['congestion_surcharge', 'airport_fee']].fillna(value = 0)

In [14]:
#Convertir columnas a tipo float.
yellow_2018_61[['congestion_surcharge', 'airport_fee']].astype(float)

,congestion_surcharge,airport_fee
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
8195670,0.0,0.0
8195671,0.0,0.0
8195672,0.0,0.0
8195673,0.0,0.0


In [15]:
#Revisar cuáles son los ejemplos de valores faltantes en 'passenger_count'.
yellow_2018_61.query("passenger_count.isnull() == True", inplace = False)

,pickup_day,pickup_time,dropoff_day,dropoff_time,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
8173487,2018-12-01,00:57:37,2018-12-01,01:20:57,NaN,6.86,90,195,0,27.36,0.00,0.5,6.79,5.76,0.3,40.71,0,0
8173488,2018-12-01,00:05:00,2018-12-01,00:28:00,NaN,13.57,22,124,0,53.75,2.75,0.5,0.00,0.00,0.3,57.30,0,0
8173489,2018-12-01,00:19:55,2018-12-01,01:15:22,NaN,16.00,230,205,0,61.50,0.00,0.5,10.21,5.76,0.3,78.27,0,0
8173490,2018-12-01,01:29:00,2018-12-01,01:52:00,NaN,6.79,237,265,0,0.00,0.00,0.0,2.00,0.00,0.3,0.00,0,0
8173491,2018-12-01,04:16:00,2018-12-01,04:32:00,NaN,8.63,170,138,0,0.00,0.00,0.5,8.91,5.76,0.3,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195670,2018-12-31,23:25:00,2018-12-31,23:40:00,NaN,9.32,50,265,0,0.00,0.00,0.0,10.00,10.50,0.3,0.00,0,0
8195671,2018-12-31,23:04:00,2018-12-31,23:30:00,NaN,9.06,243,182,0,28.95,2.75,0.5,0.00,0.00,0.3,32.50,0,0
8195672,2018-12-31,23:02:00,2018-12-31,23:35:00,NaN,11.70,236,254,0,51.05,2.75,0.5,0.00,0.00,0.3,54.60,0,0
8195673,2018-12-31,23:17:00,2018-12-31,23:43:00,NaN,7.82,146,78,0,23.19,2.75,0.5,0.00,5.76,0.3,32.50,0,0


In [16]:
#Estadísticas cuando 'passenger_count' es nulo.
yellow_2018_61.query("passenger_count.isnull() == True", inplace = False).describe()

,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,0.0,22188.000000,22188.000000,22188.000000,22188.0,22188.000000,22188.000000,22188.000000,22188.000000,22188.000000,22188.000000,22188.000000,22188.0,22188.0
mean,NaN,10.364634,137.304264,132.022399,0.0,44.087230,2.861299,0.430120,0.084365,1.830548,0.299905,49.500970,0.0,0.0
std,NaN,5.313525,74.609423,74.786864,0.0,14.630263,0.860753,0.173373,0.982657,2.862829,0.005328,15.747878,0.0,0.0
min,NaN,0.000000,3.000000,1.000000,0.0,-3.550000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,NaN,6.230000,74.000000,69.000000,0.0,32.450000,2.750000,0.500000,0.000000,0.000000,0.300000,36.000000,0.0,0.0
50%,NaN,9.720000,140.000000,131.000000,0.0,42.950000,2.750000,0.500000,0.000000,0.000000,0.300000,49.200000,0.0,0.0
75%,NaN,13.780000,210.000000,200.000000,0.0,53.750000,2.750000,0.500000,0.000000,5.760000,0.300000,60.000000,0.0,0.0
max,NaN,38.240000,265.000000,265.000000,0.0,105.950000,8.250000,0.500000,40.000000,40.000000,0.300000,112.540000,0.0,0.0


Imputar valores nulos de 'passenger_count' por el promedio redondeado a entero más cercano.

In [17]:
#Promedio de 'passenger_count'.
promedio = np.around(yellow_2018_61['passenger_count'].mean(), decimals = 0)

In [18]:
#Sustituir el promedio en los valores nulos.
yellow_2018_61['passenger_count'].fillna(promedio, inplace = True)

¿A qué se traducen PULocationID y DOLocationID?
- PULocationID: Zona en la que se activó el taxímetro.
- DOLocationID: Zona en la que se desactivó el taxímetro.

In [19]:
#Ejemplos de cantidades incluidas en PULocationID y DOLocationID.
np.sort(yellow_2018_61['PULocationID'].unique())

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 18

In [20]:
np.sort(yellow_2018_61['DOLocationID'].unique())

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 105, 106,
       107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 18

Los valores de ambas columnas van de 1 a 265 y corresponden a LocationID de taxi_zone_lookup.csv

Se rellenan medias y modas para otros posibles nulos:

In [23]:
#Calcular medias.
media_distance = np.around(yellow_2018_61['trip_distance'].mean(), decimals = 0)
media_fare = np.around(yellow_2018_61['fare_amount'].mean(), decimals = 0)
media_tip = np.around(yellow_2018_61['tip_amount'].mean(), decimals = 0)
media_tolls = np.around(yellow_2018_61['tolls_amount'].mean(), decimals = 0)
media_i_s = np.around(yellow_2018_61['improvement_surcharge'].mean(), decimals = 0)

In [24]:
#Sustituir nulos por medias.
yellow_2018_61['trip_distance'].fillna(value = media_distance, inplace = True)
yellow_2018_61['fare_amount'].fillna(value = media_fare, inplace = True)
yellow_2018_61['tip_amount'].fillna(value = media_tip, inplace = True)
yellow_2018_61['tolls_amount'].fillna(value = media_tolls, inplace = True)
yellow_2018_61['improvement_surcharge'].fillna(value = media_i_s, inplace = True)

In [25]:
#Calcular modas.
moda_payment = yellow_2018_61['payment_type'].mode().astype(int)
moda_extra = yellow_2018_61['extra'].mode().astype(int)
moda_tax = yellow_2018_61['mta_tax'].mode().astype(int)

In [26]:
#Sustituir nulos por modas.
yellow_2018_61['payment_type'].fillna(value = moda_payment, inplace = True)
yellow_2018_61['extra'].fillna(value = moda_extra, inplace = True)
yellow_2018_61['mta_tax'].fillna(value = moda_tax, inplace = True)